In [2]:
1+1

2

In [3]:
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader

# OpenAI API 키 설정 (환경변수 또는 직접 입력)
import os
from dotenv import load_dotenv

load_dotenv()

# os.environ["OPENAI_API_KEY"] = ""
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# 1. 임베딩 모델 초기화
# OpenAI의 text-embedding-ada-002 모델 사용
# 기본 1536차원 : 1536 × 4바이트 = 6.1KB per 문서
# embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 1024차원으로 축소: 512 × 4바이트 = 2.0KB per 문서 (약 67% 절약)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1024)






In [4]:
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


# 1. 문서 로더 설정 (PDF 파일 예시)
def load_and_process_document(file_path):
    """
    문서를 로드하고 청크로 분할하는 함수
    """
    # 파일 확장자에 따라 적절한 로더 선택
    if file_path.endswith(".pdf"):
        loader = PyPDFLoader(file_path)
    elif file_path.endswith(".txt"):
        loader = TextLoader(file_path, encoding="utf-8")
    else:
        raise ValueError("지원하지 않는 파일 형식입니다.")

    # 문서 로드
    documents = loader.load()

    # 2. 텍스트 분할기 설정
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,  # 각 청크의 최대 크기
        chunk_overlap=200,  # 청크 간 겹치는 부분
        length_function=len,  # 길이 계산 함수
        separators=["\n\n", "\n", " ", ""],  # 분할 기준
    )

    # 문서를 청크로 분할
    chunks = text_splitter.split_documents(documents)

    return chunks

In [9]:
import PyPDF2


def check_pdf_type(pdf_path):
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()

        if text.strip():
            print("텍스트 PDF입니다.")
        else:
            print("스캔된 이미지 PDF입니다. OCR이 필요합니다.")


check_pdf_type("sample_02.pdf")

스캔된 이미지 PDF입니다. OCR이 필요합니다.


In [18]:
import pytesseract
from pdf2image import convert_from_path
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document


def load_and_process_scanned_pdf(pdf_path):
    # PDF를 이미지로 변환
    images = convert_from_path(pdf_path)

    # 각 페이지에서 텍스트 추출
    full_text = ""
    for image in images:
        text = pytesseract.image_to_string(image, lang="kor+eng")
        full_text += text + "\n"

    # 텍스트 분할
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

    chunks = text_splitter.split_text(full_text)
    documents = [Document(page_content=chunk) for chunk in chunks]

    return documents


# 사용 예시
chunks = load_and_process_scanned_pdf("sample_02.pdf")
print(f"문서 청크 수: {len(chunks)}")

PDFInfoNotInstalledError: Unable to get page count. Is poppler installed and in PATH?

In [19]:
from pdf2image import convert_from_path

try:
    images = convert_from_path("sample_02.pdf", first_page=1, last_page=1)
    print("Poppler 설치 성공!")
except Exception as e:
    print(f"여전히 오류 발생: {e}")

여전히 오류 발생: Unable to get page count. Is poppler installed and in PATH?


In [14]:
import fitz

doc = fitz.open("sample_02.pdf")
print(f"PDF 페이지 수: {len(doc)}")

PDF 페이지 수: 5


In [8]:
# 문서가 제대로 로드되었는지 확인
chunks = load_and_process_document("sample_02.pdf")
print(f"문서 청크 수: {len(chunks)}")
print(f"첫 번째 청크 내용: {chunks[0].page_content if chunks else 'None'}")

vectorstore = Chroma.from_documents(
    documents=chunks, embedding=embeddings, persist_directory="./chroma_db"
)

type(vectorstore)

문서 청크 수: 0
첫 번째 청크 내용: None


ValueError: Expected Embeddings to be non-empty list or numpy array, got [] in upsert.

In [16]:
# 필요 패키지: pip install PyMuPDF langchain
import os
import fitz  # PyMuPDF
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter


def load_and_process_document(file_path: str):
    """
    1) PDF(=텍스트 PDF) → PyMuPDF로 페이지별 텍스트 추출
    2) TXT         → open()으로 전체 텍스트 읽기
    3) 추출한 문서를 RecursiveCharacterTextSplitter로 청크 분할
       (chunk_size=1000, overlap=200)
    반환값: List[Document]
    """
    ext = os.path.splitext(file_path)[1].lower()

    # ---------- 1. 문서 로드 ----------
    if ext == ".pdf":
        raw_docs = []
        pdf = fitz.open(file_path)

        for page_idx in range(len(pdf)):
            page = pdf.load_page(page_idx)
            text = page.get_text("text")  # 페이지의 추출 가능한 텍스트
            if not text.strip():  # 스캔 이미지 페이지 등은 건너뜀
                print("추출할 것이 없다.")
                continue

            raw_docs.append(
                Document(
                    page_content=text,
                    metadata={"source": file_path, "page": page_idx + 1},
                )
            )
        pdf.close()

    elif ext == ".txt":
        with open(file_path, encoding="utf-8") as f:
            text = f.read()

        raw_docs = [Document(page_content=text, metadata={"source": file_path})]

    else:
        raise ValueError("지원하지 않는 파일 형식입니다.")

    # ---------- 2. 텍스트 분할 ----------
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
        separators=["\n\n", "\n", " ", ""],
    )

    chunks = splitter.split_documents(raw_docs)
    return chunks


# 사용 예시 ----------------------------------------------------
chunks = load_and_process_document("sample_02.pdf")
print(f"청크 개수: {len(chunks)}")
if chunks:
    print("첫 번째 청크 내용:\n", chunks[0].page_content[:300], "...")

추출할 것이 없다.
추출할 것이 없다.
추출할 것이 없다.
추출할 것이 없다.
추출할 것이 없다.
청크 개수: 0


In [3]:
import os
import fitz  # PyMuPDF
import pytesseract
from PIL import Image
import io
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Tesseract 실행 파일 경로를 직접 지정
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

def load_and_process_document_with_ocr(file_path: str):
    """
    PDF 문서에서 텍스트를 추출합니다.
    텍스트가 없는 페이지는 이미지로 변환 후 OCR 수행.
    추출된 텍스트를 LangChain Document로 변환 후 청크 분할하여 반환.
    """
    ext = os.path.splitext(file_path)[1].lower()

    if ext != ".pdf":
        raise ValueError("현재는 PDF 파일만 지원합니다.")

    raw_docs = []
    pdf = fitz.open(file_path)

    for page_idx in range(len(pdf)):
        page = pdf.load_page(page_idx)
        text = page.get_text("text")

        if not text.strip():
            # 텍스트가 없으면 이미지로 변환 후 OCR 수행
            print(f"페이지 {page_idx + 1}: 텍스트가 없어 OCR 수행 중...")
            pix = page.get_pixmap()
            img_bytes = pix.tobytes("png")
            img = Image.open(io.BytesIO(img_bytes))
            ocr_text = pytesseract.image_to_string(img, lang="kor+eng")
            text = ocr_text

        if text.strip():
            raw_docs.append(
                Document(
                    page_content=text,
                    metadata={"source": file_path, "page": page_idx + 1},
                )
            )
        else:
            print(f"페이지 {page_idx + 1}: OCR 후에도 텍스트를 찾을 수 없습니다.")

    pdf.close()

    # 텍스트 분할
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
        separators=["\n\n", "\n", " ", ""],
    )

    chunks = splitter.split_documents(raw_docs)
    return chunks


# 사용 예시
chunks = load_and_process_document_with_ocr("sample_02.pdf")
print(f"청크 개수: {len(chunks)}")
if chunks:
    print("첫 번째 청크 내용:\n", chunks[0].page_content[:300], "...")

페이지 1: 텍스트가 없어 OCR 수행 중...
페이지 2: 텍스트가 없어 OCR 수행 중...
페이지 3: 텍스트가 없어 OCR 수행 중...
페이지 4: 텍스트가 없어 OCR 수행 중...
페이지 5: 텍스트가 없어 OCR 수행 중...
청크 개수: 5
첫 번째 청크 내용:
 정책 기본 정보

정책명

정책소개

정책 분야

지원 내용

사업 운영 기간
사업 신청 기간
지원 규모(명)

비고

청년정책 정보

첨년일자리 도약장려금

기업의 청년고용 SHS 지원하고, 취업애로 청년의 취업을 촉진함으로써,
ADS 활성화를 목적으로 하는 정책

0 (유형 1) 5인 이상 중소기업에서 취업애로청년을 정규직으로 채용하고
oR 이상 고용유지시 기업지원금 지급

ㅇ (유형) 빈일자리 업종 중소기업에서 청년을 정규직으로 채용 후 6개월
이상 고용유지시 기업지원금 지급하고, 해당 기업에서 18개월 이상 재직한
첨년에게  ...
